# function calling 

In [1]:
import os
import openai
from dotenv import load_dotenv

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = OPENAI_API_KEY


In [3]:
prompt = "Hello, How are you?"

response = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]
)

print(response["choices"][0]["message"]["content"])

Hello! I'm just a computer program, so I don't have feelings, but I'm here to assist you. How can I help you today?


In [4]:
import requests
def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

    querystring = {"text":location}

    headers = {
      'x-rapidapi-key': "156877d1e1msh6806b57dfec44b9p1bd2c8jsn796f2ff15790",
      'x-rapidapi-host': "ai-weather-by-meteosource.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())
  
    return response.json()

In [5]:
response=get_current_weather("bangalore")

[{'name': 'Bengaluru', 'place_id': 'bengaluru', 'adm_area1': 'Karnataka', 'adm_area2': 'Bengaluru Urban', 'country': 'India', 'lat': '12.97194N', 'lon': '77.59369E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Bengaluru Urban', 'place_id': 'bangalore-urban', 'adm_area1': 'Karnataka', 'adm_area2': 'Bengaluru Urban', 'country': 'India', 'lat': '13.0N', 'lon': '77.58333E', 'timezone': 'Asia/Kolkata', 'type': 'administrative_area'}, {'name': 'Yelahanka', 'place_id': 'yelahanka', 'adm_area1': 'Karnataka', 'adm_area2': 'Bengaluru Urban', 'country': 'India', 'lat': '13.10073N', 'lon': '77.59632E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Anekal', 'place_id': 'anekal', 'adm_area1': 'Karnataka', 'adm_area2': 'Bengaluru Urban', 'country': 'India', 'lat': '12.7111N', 'lon': '77.69557E', 'timezone': 'Asia/Kolkata', 'type': 'settlement'}, {'name': 'Devanahalli', 'place_id': 'devanhalli', 'adm_area1': 'Karnataka', 'adm_area2': 'Bengaluru Rural', 'country': 'India

In [6]:
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    
                },
                "required": ["location"],
            },
        }
    ]

In [7]:


user_message="Hi There"
messages=[]
messages.append({"role": "user", "content":user_message})

completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages= messages
    
)

In [8]:
completion.choices[0].message

<OpenAIObject at 0x1f0c8025f70> JSON: {
  "role": "assistant",
  "content": "Hello! How can I assist you today?",
  "refusal": null
}

In [9]:
messages

[{'role': 'user', 'content': 'Hi There'}]

In [10]:
user_message="What is the temperature of Bangalore"

messages.append({"role": "user", "content": user_message})

completion=openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
    
)

In [11]:
print(completion.choices[0].message)

{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"Bangalore\"}"
  },
  "refusal": null
}


In [12]:
response=completion.choices[0].message

In [13]:
response

<OpenAIObject at 0x1f0c8025df0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\"location\":\"Bangalore\"}"
  },
  "refusal": null
}

In [14]:
function_name=response['function_call']['name']
print(function_name)


get_current_weather


In [15]:
import json
location=eval(response['function_call']['arguments'])['location']
print(location)

Bangalore


In [16]:
messages.append(response)  # extend conversation with assistant's reply
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": location,
    }
)

In [17]:
messages

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user', 'content': 'What is the temperature of Bangalore'},
 <OpenAIObject at 0x1f0c8025df0> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\"location\":\"Bangalore\"}"
   },
   "refusal": null
 },
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Bangalore'}]

In [18]:
# extend conversation with function response
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions
)  # get a new response from GPT where it can see the function response


In [19]:
print(second_response.choices[0].message)

{
  "role": "assistant",
  "content": "The current temperature in Bangalore is 26\u00b0C.",
  "refusal": null
}


In [20]:
print("The End")

The End
